In [5]:
from dotenv import load_dotenv
import os

from eodhd import APIClient
import json

In [55]:
load_dotenv()
api_token = "demo"
api = APIClient(api_token)

In [56]:
historical_data = api.get_historical_data("TSLA", "d", "2023-10-1", "2024-09-01")

In [ ]:
print(historical_data)

In [57]:
fundamental_data = api.get_fundamentals_data("TSLA.US")

In [ ]:
print(fundamental_data)

In [58]:
news_data = api.financial_news(s="TSLA.US", from_date="2023-09-01", to_date="2024-09-02", limit=100)

In [ ]:
print(news_data)